Note: Multiplier has been taken as it is. For averages and ratios it doesn't affect athe analysis. To use absolute population values, use multiplier/100 instead

# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pathlib
import pickle
from functools import reduce

In [2]:
pd.set_option('display.max_columns', None)

# Importing datasets

In [3]:
# Checking paths
pathlib.Path("../../HCES 2022-23/Python implementation/Codes/Population based MPCE/MPCE_1Jan24.pkl").resolve().exists()

True

In [4]:
# Reading HH-wise MPCE
path_mpce = "../../HCES 2022-23/Python implementation/Codes/Population based MPCE/MPCE_1Jan24.pkl"
df_mpce = pd.read_pickle(path_mpce)
df_mpce = df_mpce[['hh_size', 'sector', 'mpce', 'decile']]
display(df_mpce.head())

## Level 01 file
path_lvl01 = "../../HCES 2022-23/Dta raw files/level_01.dta"
df_lvl01 = pd.read_stata(path_lvl01)
df_lvl01 = df_lvl01.set_index('hhid')
df_lvl01 = df_lvl01[['state']]
display(df_lvl01.head())

## Level 03 file
path_lvl03 = "../../HCES 2022-23/Dta raw files/level_03.dta"
df_lvl03 = pd.read_stata(path_lvl03)
df_lvl03 = df_lvl03.set_index('hhid')
df_lvl03 = df_lvl03[['source_cooking', 'multiplier']]
display(df_lvl03.head())

## Level 07 file
path_lvl07 = "../../HCES 2022-23/Dta raw files/level_07.dta"
df_lvl07 = pd.read_stata(path_lvl07)
df_lvl07 = df_lvl07.set_index('hhid')
df_lvl07 = df_lvl07[['receieved_subsidy_lpg', 'num_subsidized_lpg']]
display(df_lvl07.head())

## Level 08 file
path_lvl08 = "../../HCES 2022-23/Dta raw files/level_08.dta"
df_lvl08 = pd.read_stata(path_lvl08)
df_lvl08 = df_lvl08.loc[df_lvl08['item_code'].isin([331,338])]

,hh_size,sector,mpce,decile
hhid,,,,
31000301,1.0,2,19675.64,10
31000302,5.0,2,8059.52,8
31000303,5.0,2,7628.02,8
31000304,2.0,2,6522.62,7
31000305,1.0,2,6765.77,7


,state
hhid,
65556201,1
65556301,1
65556302,1
65556303,1
65556304,1


,source_cooking,multiplier
hhid,,
31000301,2,96498
31000302,2,96498
31000303,2,96498
31000304,2,96498
31000305,2,96498


,receieved_subsidy_lpg,num_subsidized_lpg
hhid,,
31000301,2,NaN
31000302,1,1.0
31000303,1,1.0
31000304,1,1.0
31000305,2,NaN


# Data wrangling

In [5]:
# Reshaping level 08 file for firewood and LPG

## Pivoting
df_lvl08 = df_lvl08.pivot_table(index = 'hhid', columns = 'item_code', values = ['cons_total_qty','cons_total_value', 'cons_home_qty', 'cons_home_value' ])

## Converting columns to single level index
df_lvl08.columns = ["_".join(map(str, x)) for x in df_lvl08.columns]

## Changing column names
df_lvl08 = df_lvl08.rename(columns = {'cons_total_qty_331': 'firewood_qty', 'cons_total_qty_338': 'lpg_qty',
                               'cons_total_value_331': 'firewood_value', 'cons_total_value_338': 'lpg_value',
                               'cons_home_qty_331': 'firewood_home_qty', 'cons_home_value_331': 'firewood_home_value',
                               'cons_home_qty_338': 'lpg_home_qty', 'cons_home_value_338': 'lpg_home_value'})
df_lvl08.head()

# Adding market quantity and value
df_lvl08 = df_lvl08.fillna(0)
df_lvl08[['firewood_mkt_qty', 'firewood_mkt_value']] = df_lvl08[['firewood_qty', 'firewood_value' ]].values - df_lvl08[['firewood_home_qty', 'firewood_home_value' ]].values
df_lvl08[['lpg_mkt_qty', 'lpg_mkt_value']] = df_lvl08[['lpg_qty', 'lpg_value' ]].values - df_lvl08[['lpg_home_qty', 'lpg_home_value' ]].values

df_lvl08.head()

,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,
31000301,0.0,0.0,0.0,0.0,0.0,2.0,0.0,240.0,0.0,0.0,2.0,240.0
31000302,0.0,0.0,0.0,0.0,0.0,8.0,0.0,640.0,0.0,0.0,8.0,640.0
31000303,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
31000304,0.0,0.0,0.0,0.0,0.0,5.0,0.0,400.0,0.0,0.0,5.0,400.0
31000305,0.0,0.0,0.0,0.0,0.0,4.0,0.0,320.0,0.0,0.0,4.0,320.0


In [6]:
# Merging all files

## Checking shapes
print(f"Shape of MPCE file: {df_mpce.shape}")
print(f"Shape of level 01 file: {df_lvl01.shape}")
print(f"Shape of level 03 file: {df_lvl03.shape}")
print(f"Shape of level 07 file: {df_lvl07.shape}")
print(f"Shape of level 08 file: {df_lvl08.shape}")

## Defining merge function
def merge_df (left_df, right_df):
    df = pd.merge(left_df, right_df, how = 'outer', left_index=True, right_index=True, indicator=True)
    print(f"\nSummary from merging dataframes")
    display(df['_merge'].value_counts())
    df = df.drop(labels = '_merge', axis = 1)
    return df.copy()

## Merging
df = reduce(merge_df, [df_mpce, df_lvl01, df_lvl03, df_lvl07, df_lvl08])

Shape of MPCE file: (261746, 4)
Shape of level 01 file: (261746, 1)
Shape of level 03 file: (261746, 2)
Shape of level 07 file: (261746, 2)
Shape of level 08 file: (252955, 12)

Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          252955
left_only       8791
right_only         0
Name: count, dtype: int64

In [7]:
# Renaming column values
df['sector'] = df['sector'].replace({1: 'rural', 2: 'urban'})
df['state'] = df['state'].replace({1: 'Jammu & Kashmir', 2: 'Himachal Pradesh', 3: 'Punjab', 4: 'Chandigarh(U.T.)', 5: 'Uttrakhand', 6: 'Haryana', 7: 'Delhi', 8: 'Rajasthan', 9: 'Uttar Pradesh', 10: 'Bihar', 11: 'Sikkim', 12: 'Arunachal Pradesh', 13: 'Nagaland', 14: 'Manipur', 15: 'Mizoram', 16: 'Tripura', 17: 'Meghalaya', 18: 'Assam', 19: 'West Bengal', 20: 'Jharkhand', 21: 'Odisha', 22: 'Chattisgarh', 23: 'Madhya Pradesh', 24: 'Gujarat', 25: 'Dadra & Nagar Haveli', 27: 'Maharashtra', 28: 'Andhra Pradesh', 29: 'Karnataka', 30: 'Goa', 31: 'Lakshadweep (U.T.)', 32: 'Kerala', 33: 'Tamilnadu', 34: 'Puducherry (U.T.)', 35: 'A and N Islands (U.T.)', 36: 'Telangana', 37: 'Ladakh (U.T.)'})
df['source_cooking'] = df['source_cooking'].replace({1: "firewood and chips", 2: "LPG", 3: "other natural gas", 4: "dung cake", 5: "kerosene", 6: "coke, coal", 7: "gobar gas", 8: "other biogas", 9: "others", 10: "charcoal", 11: "electricity", 12: "no cooking arrangement"})
df['receieved_subsidy_lpg'] = df['receieved_subsidy_lpg'].replace({1: 'yes', 2: 'no'})

# Analysis

## Primary cooking source

In [8]:
# Across India

## Calculating
trial = df.groupby(['source_cooking'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,source_cooking,popn_size,sample_size
0,LPG,15927952229,3206580
1,charcoal,2740225,1700
2,"coke, coal",101807090,23120
3,dung cake,695334826,101520
4,electricity,35922157,8620
5,firewood and chips,10649712107,1738940
6,gobar gas,13655523,2100
7,kerosene,19559832,4120
8,no cooking arrangement,628452832,76020
9,other biogas,2135539,300


In [9]:
# Primary cooking source (Rural vs Urban)

## Calculating
trial = df.groupby(['sector','source_cooking']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
trial = trial.reset_index()

## Pivoting
trial = trial.pivot_table(values = 'popn_size', index = ['sector'], columns = 'source_cooking')
display(trial)

source_cooking,LPG,charcoal,"coke, coal",dung cake,electricity,firewood and chips,gobar gas,kerosene,no cooking arrangement,other biogas,other natural gas,others
sector,,,,,,,,,,,,
rural,8.366512e+09,1060130.0,55011889.0,673427775.0,9290719.0,1.006922e+10,13407987.0,3408425.0,211530273.0,1654981.0,24215703.0,60796558.0
urban,7.561440e+09,1680095.0,46795201.0,21907051.0,26631438.0,5.804880e+08,247536.0,16151407.0,416922559.0,480558.0,243365178.0,34192124.0


In [10]:
# Primary cooking source (Rural vs Urban by decile)

## Calculating
trial = df.groupby(['sector','decile','source_cooking'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)


## Pivoting
trial = trial.pivot_table (values = 'popn_size', index = ['sector', 'source_cooking'], columns = 'decile', observed = False)
display(trial)


decile                                    1             2             3  \
sector source_cooking                                                     
rural  LPG                     1.739902e+08  3.074245e+08  4.237089e+08   
       charcoal                         NaN  2.671700e+04  1.160180e+05   
       coke, coal              4.953282e+06  6.149718e+06  4.852760e+06   
       dung cake               1.111776e+08  8.653105e+07  8.559475e+07   
       electricity             1.238450e+05  4.391210e+05  5.553820e+05   
       firewood and chips      1.154369e+09  1.174058e+09  1.145904e+09   
       gobar gas               1.229120e+05  5.650610e+05  3.969680e+05   
       kerosene                1.023760e+05  2.253160e+05  5.854030e+05   
       no cooking arrangement  2.202948e+07  6.703582e+06  5.854034e+06   
       other biogas                     NaN           NaN           NaN   
       other natural gas       1.438760e+05  3.800080e+05  2.456530e+05   
       others                  3.254250e+06  6.893897e+06  5.747299e+06   
urban  LPG                     3.879099e+08  5.351827e+08  6.209003e+08   
       charcoal                6.174370e+05  8.257200e+04  7.134100e+04   
       coke, coal              1.736651e+07  9.697889e+06  7.248269e+06   
       dung cake               8.427711e+06  4.144291e+06  3.749510e+06   
       electricity             1.039372e+06  1.223064e+06  9.829890e+05   
       firewood and chips      1.748448e+08  1.149806e+08  8.166183e+07   
       gobar gas               1.243280e+05           NaN           NaN   
       kerosene                2.649891e+06  1.996846e+06  1.537280e+06   
       no cooking arrangement  4.901976e+06  1.098210e+07  9.612392e+06   
       other biogas                     NaN           NaN  1.002720e+05   
       other natural gas       5.864740e+05  2.409157e+06  6.622422e+06   
       others                  1.978457e+06  1.267252e+06  1.818423e+06   

decile                                    4             5             6  \
sector source_cooking                                                     
rural  LPG                     5.167786e+08  6.582871e+08  7.927863e+08   
       charcoal                         NaN  4.020000e+04  5.760900e+04   
       coke, coal              5.822642e+06  6.715582e+06  7.928990e+06   
       dung cake               6.595925e+07  6.574165e+07  5.870363e+07   
       electricity             5.490570e+05  8.426650e+05  6.634990e+05   
       firewood and chips      1.146830e+09  1.067373e+09  1.048291e+09   
       gobar gas               6.416330e+05  6.364060e+05  1.269761e+06   
       kerosene                2.632240e+05  2.047520e+05  2.806350e+05   
       no cooking arrangement  7.974438e+06  1.165704e+07  9.612868e+06   
       other biogas            1.461830e+05           NaN           NaN   
       other natural gas       3.493560e+05  5.964800e+04  5.885920e+05   
       others                  5.494138e+06  8.585061e+06  5.562990e+06   
urban  LPG                     6.775526e+08  7.339306e+08  7.974095e+08   
       charcoal                1.833760e+05  4.982000e+03  1.418900e+05   
       coke, coal              4.550756e+06  2.924138e+06  2.281678e+06   
       dung cake               2.082957e+06  1.249871e+06  8.898190e+05   
       electricity             8.169010e+05  1.002746e+06  9.110020e+05   
       firewood and chips      5.986677e+07  4.568273e+07  3.682873e+07   
       gobar gas                        NaN           NaN           NaN   
       kerosene                1.020016e+06  2.194275e+06  1.376401e+06   
       no cooking arrangement  7.982244e+06  8.785994e+06  9.683630e+06   
       other biogas            1.119490e+05  2.683370e+05           NaN   
       other natural gas       7.829040e+06  1.233375e+07  1.697082e+07   
       others                  2.564539e+06  1.250877e+06  1.724240e+06   

decile                                   7             8             9  \
sector source_cooking   

## LPG subsidy - Last three months

In [11]:
# Across India
trial = df.groupby(['receieved_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,receieved_subsidy_lpg,popn_size,sample_size
0,no,22129917998,4065540
1,yes,6309923925,1169380


In [12]:
# LPG subsidy (Rural vs Urban)
trial = df.groupby(['sector','receieved_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)

# Pivoting 
trial = trial.pivot_table(values = 'popn_size', index = 'receieved_subsidy_lpg', columns = 'sector')
display(trial)

sector,rural,urban
receieved_subsidy_lpg,,
no,1.533033e+10,6.799583e+09
yes,4.159206e+09,2.150718e+09


In [13]:
# LPG subsidy (Rural vs Urban by decile)
trial = df.groupby(['sector','decile','receieved_subsidy_lpg'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'wt_sum': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)

# Pivoting
trial = trial.pivot_table(values = 'wt_sum', index = ['sector','receieved_subsidy_lpg'], columns = 'decile', observed = False)
display(trial)

decile                                   1             2             3  \
sector receieved_subsidy_lpg                                             
rural  no                     1.345338e+09  1.351555e+09  1.375220e+09   
       yes                    1.249284e+08  2.378424e+08  2.983407e+08   
urban  no                     4.426189e+08  4.898582e+08  5.283760e+08   
       yes                    1.578280e+08  1.921082e+08  2.059290e+08   

decile                                   4             5             6  \
sector receieved_subsidy_lpg                                             
rural  no                     1.429463e+09  1.447575e+09  1.498177e+09   
       yes                    3.213452e+08  3.725681e+08  4.275693e+08   
urban  no                     5.578999e+08  5.975515e+08  6.431201e+08   
       yes                    2.066612e+08  2.120767e+08  2.250977e+08   

decile                                   7             8             9  \
sector receieved_subsidy_lpg                                             
rural  no                     1.533765e+09  1.623491e+09  1.679853e+09   
       yes                    4.804035e+08  5.311480e+08  6.326242e+08   
urban  no                     6.942477e+08  7.553499e+08  8.458141e+08   
       yes                    2.251614e+08  2.399774e+08  2.613067e+08   

decile                                  10  
sector receieved_subsidy_lpg                
rural  no                     2.045896e+09  
       yes                    7.324366e+08  
urban  no                     1.244747e+09  
       yes                    2.245714e+08

# LPG susbsidised cylinder - Last 3 months

In [14]:
# Across India

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby('num_subsidized_lpg', as_index = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,num_subsidized_lpg,popn_size,sample_size
0,1.0,3900515861,688320
1,2.0,1750466336,337620
2,3.0,658941728,143440


In [15]:
# Rural vs Urban

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby(['sector','num_subsidized_lpg']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

popn_size  sample_size
sector num_subsidized_lpg                         
rural  1.0                 2831518859       410514
       2.0                 1003612412       148789
       3.0                  324075030        50312
urban  1.0                 1068997002       243390
       2.0                  746853924       171950
       3.0                  334866698        85956

In [16]:
# Rural vs Urban across deciles

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

## Calculation
trial = trial.groupby(['sector', 'decile', 'num_subsidized_lpg'], observed = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
trial = trial.reset_index()

# Pivotting
trial = trial.pivot_table(values = 'popn_size', index = ['sector', 'num_subsidized_lpg'], columns = 'decile',observed = False)
display(trial)

decile                              1            2            3            4  \
sector num_subsidized_lpg                                                      
rural  1.0                 97305500.0  178581965.0  219297280.0  232676510.0   
       2.0                 21215479.0   45025801.0   59989360.0   64605908.0   
       3.0                  6407458.0   14234600.0   19054099.0   24062738.0   
urban  1.0                 77707959.0   91110693.0  104067025.0  103230337.0   
       2.0                 52677297.0   66364220.0   67085649.0   70094292.0   
       3.0                 27442744.0   34633269.0   34776332.0   33336564.0   

decile                               5            6            7            8  \
sector num_subsidized_lpg                                                       
rural  1.0                 261046305.0  299532810.0  315431768.0  354278978.0   
       2.0                  84526979.0   95695982.0  120638223.0  138301253.0   
       3.0                  26994792.0   32340486.0   44333482.0   38567749.0   
urban  1.0                 104273686.0  109300637.0  109154327.0  119131765.0   
       2.0                  75858431.0   80691239.0   80106084.0   83525653.0   
       3.0                  31944616.0   35105785.0   35900946.0   37319979.0   

decile                               9           10  
sector num_subsidized_lpg                            
rural  1.0                 409245667.0  464122076.0  
       2.0                 170929653.0  202683774.0  
       3.0                  52448904.0   65630722.0  
urban  1.0                 135015828.0  116004745.0  
       2.0                  91618712.0   78832347.0  
       3.0                  34672132.0   29734331.0

# Average quantity and value of TOTAL LPG and firewood consumed per capita

* Values are reported for past 30 days

In [8]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum(),
    'lpg_value':(trial['lpg_value'] * trial['multiplier']).sum(),
    'firewood_qty': (trial['firewood_qty'] * trial['multiplier']).sum(),
    'firewood_value':(trial['firewood_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

lpg_qty             1.473315
lpg_value         117.535183
firewood_qty       11.037265
firewood_value     51.857151
dtype: float64

In [243]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_lpg_qty,wt_lpg_value,wt_firewod_qty,wt_firewood_value
sector,,,,
rural,1.127155,90.514766,14.474689,67.810782
urban,2.348439,185.845210,2.347152,11.524946


In [247]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0).round(2)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewod_qty  wt_firewood_value
sector decile                                                             
rural  1             0.29         23.72           15.31              67.78
       2             0.52         41.90           16.06              72.38
       3             0.68         54.98           15.78              72.80
       4             0.80         64.15           15.65              73.38
       5             0.97         77.81           15.17              71.79
       6             1.13         90.41           14.99              71.43
       7             1.30        104.57           14.19              67.55
       8             1.51        120.98           13.87              67.05
       9             1.78        142.92           12.60              60.58
       10            2.29        183.72           11.13              53.38
urban  1             1.38        109.59            6.13              29.79
       2             1.84        145.65            4.30              20.93
       3             2.08        164.73            3.16              15.70
       4             2.23        176.09            2.56              12.93
       5             2.35        185.86            2.06               9.91
       6             2.51        198.28            1.65               8.40
       7             2.60        205.14            1.35               6.83
       8             2.72        215.05            0.92               4.39
       9             2.84        225.40            0.76               3.83
       10            2.93        232.67            0.58               2.54

In [11]:
df.head()

,hh_size,sector,mpce,decile,state,source_cooking,multiplier,receieved_subsidy_lpg,num_subsidized_lpg,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,,,,,,,,,,
31000301,1.0,urban,19675.64,10,Andhra Pradesh,LPG,96498,no,NaN,0.0,0.0,0.0,0.0,0.0,2.0,0.0,240.0,0.0,0.0,2.0,240.0
31000302,5.0,urban,8059.52,8,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,640.0,0.0,0.0,8.0,640.0
31000303,5.0,urban,7628.02,8,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
31000304,2.0,urban,6522.62,7,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,400.0,0.0,0.0,5.0,400.0
31000305,1.0,urban,6765.77,7,Andhra Pradesh,LPG,96498,no,NaN,0.0,0.0,0.0,0.0,0.0,4.0,0.0,320.0,0.0,0.0,4.0,320.0


In [12]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty'] * table['multiplier']).sum(),
    'wt_lpg_value':(table['lpg_value'] * table['multiplier']).sum(),
    'wt_firewood_qty': (table['firewood_qty'] * table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
rural  A and N Islands (U.T.)    2.655244    217.624077         2.361476   
       Andhra Pradesh            1.895074    152.515515         6.706925   
       Arunachal Pradesh         0.968536     79.500800        25.564439   
       Assam                     0.967616     78.142976        16.955429   
       Bihar                     0.854335     72.119167        10.485264   
...                                   ...           ...              ...   
urban  Telangana                 2.014468    168.378900         0.260950   
       Tripura                   2.711926    239.896157         7.097235   
       Uttar Pradesh             2.300679    181.358964         1.678701   
       Uttrakhand                2.824488    221.467120         1.627196   
       West Bengal               2.640153    208.327769         4.619056   

                               wt_firewood_value  
sector state                                      
rural  A and N Islands (U.T.)          14.484645  
       Andhra Pradesh                  27.148969  
       Arunachal Pradesh              140.625085  
       Assam                           84.168833  
       Bihar                           69.381512  
...                                          ...  
urban  Telangana                        1.722246  
       Tripura                         45.793740  
       Uttar Pradesh                    8.900367  
       Uttrakhand                       6.784211  
       West Bengal                     23.653839  

[72 rows x 4 columns]

In [14]:
trial[trial.index.get_level_values(0) == 'rural']

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
rural  A and N Islands (U.T.)    2.655244    217.624077         2.361476   
       Andhra Pradesh            1.895074    152.515515         6.706925   
       Arunachal Pradesh         0.968536     79.500800        25.564439   
       Assam                     0.967616     78.142976        16.955429   
       Bihar                     0.854335     72.119167        10.485264   
       Chandigarh(U.T.)          2.308463    179.290253         0.022060   
       Chattisgarh               0.506296     42.404400        21.453751   
       Dadra & Nagar Haveli      1.744730    145.776756        12.013179   
       Delhi                     2.660327    203.502311         0.121473   
       Goa                       2.734743    210.097285         2.931283   
       Gujarat                   1.020756     78.678807        13.605147   
       Haryana                   1.653229    127.356067         5.411886   
       Himachal Pradesh          1.620603    131.256653        34.140883   
       Jammu & Kashmir           1.417372    117.218134        14.437701   
       Jharkhand                 0.350521     28.844461        22.118967   
       Karnataka                 1.599367    126.549188        11.897601   
       Kerala                    1.867357    148.988925        19.000554   
       Ladakh (U.T.)             1.424168    128.181582         2.885705   
       Lakshadweep (U.T.)        1.704967    187.612650         8.055323   
       Madhya Pradesh            0.559659     44.681054        15.626320   
       Maharashtra               1.524619    120.678786        10.982248   
       Manipur                   1.445872    126.261642         5.521463   
       Meghalaya                 0.432300     36.466339        21.390110   
       Mizoram                   3.440242    238.470053         6.456494   
       Nagaland                  0.747715     64.437590        52.378334   
       Odisha                    0.615393     48.782110        35.119628   
       Puducherry (U.T.)         2.569520    202.817792         2.033611   
       Punjab                    2.318546    184.615732         4.019395   
       Rajasthan                 0.836019     64.987557        21.415692   
       Sikkim                    1.860330    174.453921        17.287447   
       Tamilnadu                 2.147863    172.127640         7.302085   
       Telangana                 1.541588    129.171543         3.606624   
       Tripura                   0.951456     84.481322        23.608431   
       Uttar Pradesh             1.017101     81.774805        11.302356   
       Uttrakhand                1.775970    141.067433        13.377306   
       West Bengal               0.838022     66.456505        23.068512   

                               wt_firewood_value  
sector state                                      
rural  A and N Islands (U.T.)          14.484645  
       Andhra Pradesh                  27.148969  
       Arunachal Pradesh              140.625085  
       Assam                           84.168833  
       Bihar                           69.381512  
       Chandigarh(U.T.)                 0.671203  
       Chattisgarh                    110.025868  
       Dadra & Nagar Haveli            64.897730  
       Delhi                            0.999543  
       Goa                             13.099684  
       Gujarat                         69.638123  
       Haryana                         30.775406  
       Himachal Pradesh               137.734640  
       Jammu & Kashmir                 50.355879  
       Jharkhand                       96.478444  
       Karnataka                       54.079397  
       Kerala                          75.806149  
       Ladakh (U.T.)                   22.891905  
       Lakshadweep (U.T.)              46.039432  
       Madhya Pradesh                  62.527672  
       Maharashtra  

# Average quantity and value of MARKET firewood consumed per capita

In [255]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'firewood_mkt_qty': (trial['firewood_mkt_qty'] * trial['multiplier']).sum(),
    'firewood_mkt_value':(trial['firewood_mkt_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

firewood_mkt_qty       7.677946
firewood_mkt_value    36.449999
dtype: float64

In [256]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_firewood_mkt_qty,wt_firewood_mkt_value
sector,,
rural,9.986364,47.110268
urban,1.842060,9.499887


In [259]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector decile                                            
rural  1                 12.019989              53.076220
       2                 11.860949              53.813415
       3                 11.320030              52.788773
       4                 11.180783              52.685019
       5                 10.659816              50.925218
       6                 10.225955              48.933431
       7                  9.329608              44.787533
       8                  9.058715              44.088838
       9                  7.701046              37.614000
       10                 6.506182              32.387946
urban  1                  5.301535              26.299058
       2                  3.505134              17.804532
       3                  2.401369              12.651682
       4                  1.930987              10.328211
       5                  1.585764               7.959233
       6                  1.203891               6.656743
       7                  0.990697               5.384606
       8                  0.592732               3.191518
       9                  0.532468               2.946313
       10                 0.375457               1.774245

In [40]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
rural  A and N Islands (U.T.)             2.286046              13.300046
       Andhra Pradesh                     6.404175              25.909515
       Arunachal Pradesh                 19.685340             112.403346
       Assam                              8.646849              46.328774
       Bihar                              7.647358              49.365340
...                                            ...                    ...
urban  Telangana                          0.247391               1.633835
       Tripura                            5.669061              37.511397
       Uttar Prdesh                       1.403836               7.702412
       Uttrakhand                         1.446670               6.368216
       West Bengal                        4.026102              20.790609

[72 rows x 2 columns]

In [41]:
trial[trial.index.get_level_values(0) == 'urban']

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
urban  A and N Islands (U.T.)             0.017689               0.106132
       Andhra Pradesh                     0.691379               3.956743
       Arunachal Pradesh                  5.820248              36.357900
       Assam                              1.559906               9.377244
       Bihar                              2.334052              15.699532
       Chandigarh(U.T.)                   0.000000               0.000000
       Chattisgarh                        3.835990              19.932021
       Dadra & Nagar Haveli               1.633755               8.175905
       Delhi                              0.009241               0.146277
       Goa                                2.292424              10.585234
       Gujarat                            0.883754               5.419045
       Haryana                            0.736072               4.591903
       Himachal Pradesh                   2.526846               9.281232
       Jammu & Kashmir                    1.411953               6.207606
       Jharkhand                          2.997611              13.316326
       Karnataka                          1.815948               9.130688
       Kerala                             5.917278              26.378883
       Ladakh (U.T.)                      0.342992               2.826118
       Lakshadweep (U.T.)                 1.317188               4.160291
       Madhya Pradesh                     1.991260               9.375832
       Maharashtra                        0.763228               3.927712
       Manipur                            0.496880               3.986117
       Meghalaya                          2.909208              13.123720
       Mizoram                            0.076945               0.349433
       Nagaland                          12.792699              54.468782
       Odisha                            11.647648              51.931643
       Puducherry (U.T.)                  1.310784               8.268590
       Punjab                             1.102725               8.503563
       Rajasthan                          2.284589              12.119382
       Sikkim                             0.201413               0.604240
       Tamilnadu                          0.836909               4.823335
       Telangana                          0.247391               1.633835
       Tripura                            5.669061              37.511397
       Uttar Prdesh                       1.403836               7.702412
       Uttrakhand                         1.446670               6.368216
       West Bengal                        4.026102              20.790609

# Household average annual cylinder consumption

In [ ]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum()/trial['multiplier'].sum(),
})

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

lpg_qty    5.31
dtype: float64

In [ ]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False)

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

,wt_lpg_qty
sector,
rural,4.25
urban,7.63


In [34]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum()
    }), include_groups = False).round(2)

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

wt_lpg_qty
sector decile            
rural  1             1.47
       2             2.40
       3             3.00
       4             3.35
       5             3.91
       6             4.29
       7             4.75
       8             5.15
       9             5.67
       10            6.06
urban  1             6.70
       2             7.85
       3             8.26
       4             8.48
       5             8.46
       6             8.39
       7             8.21
       8             7.94
       9             7.45
       10            5.81